# Step One: Generating a script for the video based on a sentence

First: run this command to get the model: 

Non Instruct:
```
optimum-cli export openvino --model meta-llama/Llama-3.2-1B --task text-generation-with-past ./Llama-3.2-1B_openvino
```

Instruct:
```
optimum-cli export openvino --model meta-llama/Llama-3.2-1B-Instruct --task text-generation-with-past ./Llama-3.2-1B-Instruct_openvino
```

With Quanitization: 

Non Instruct:
```
optimum-cli export openvino --model meta-llama/Llama-3.2-1B --task text-generation-with-past --weight-format int8 ./Llama-3.2-1B_openvino_INT8
```

Instruct:
```
optimum-cli export openvino --model meta-llama/Llama-3.2-1B-Instruct --task text-generation-with-past --weight-format int8 ./Llama-3.2-1B-Instruct_openvino_INT8
```


In [1]:
from transformers import AutoTokenizer, AutoConfig
from optimum.intel.openvino import OVModelForCausalLM
import openvino.runtime as ov

# Path to the converted model
model_dir = "./Llama-3.2-1B-instruct_openvino_INT8"

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# Select device (CPU or GPU)
device = "CPU"  # Change to "GPU" if available and desired


# Initialize model
model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)


/home/intelaipc/intel/oneapi/intelpython/envs/tiktalk/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/intelaipc/intel/oneapi/intelpython/envs/tiktalk/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Compiling the model to CPU ...


In [2]:
import torch

def generate_chat_response(prompt: str,
                          max_new_tokens: int = 256,
                          temperature: float = 1.0,
                          top_p: float = 0.9,
                          top_k: int = 50,
                          repetition_penalty: float = 1.2) -> str:
    """
    Generates a chat response using the Llama-3.2-1B-Instruct model.

    Parameters:
        prompt (str): The input string for the chatbot.
        max_new_tokens (int): Maximum number of tokens to generate.
        temperature (float): Sampling temperature.
        top_p (float): Nucleus sampling parameter.
        top_k (int): Top-K sampling parameter.
        repetition_penalty (float): Repetition penalty.

    Returns:
        str: The generated response.
    """
    # Tokenize the input prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Generate response
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            do_sample=True
        )
    
    # Decode the output tokens
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    # Optionally, remove the prompt from the response
    if response.startswith(prompt):
        response = response[len(prompt):].strip()
    
    return response


In [3]:
prompt = """\n\n
Your task is to create a 30 second engaging and educational tiktok script based on the following sentence:

{input_sentence}

Expand on this sentence to create an interesting and educational script that most people might not know about.
The tiktok should incorporate an engaging story or example related to the sentence.
Do not include any emojis or hashtags in the script.
The script should be only spoken text, no extra text like [Cut] or [Music].
The script should sound passionate, excited, and happy.

Script:
"""

sentence = "Spaceships are the future of human travel."

user_input = prompt.format(input_sentence=sentence)


In [4]:
response = generate_chat_response(user_input, max_new_tokens=1000)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"Imagine living on Mars one day - it's already happening. In fact, NASA has been sending astronauts all over space for decades. But what you may not know is how they're doing exactly as planned."

"But did you know they have to wait years between launches? Because each launch costs millions dollars per seat! That means every dollar going towards building new spaceships and spacecraft is contributing towards making humanity multiplanetary." 

"You see, our planet Earth was facing extinction from asteroids and other dangers, but now we can't let another disaster happen while trying to explore the cosmos with just two small planets nearby: earth and mars." "It could get ugly very fast if there aren't enough resources available elsewhere so humans have made agreements such as the Artemis Program which would help us acquire more resources before being able to establish itself officially"


In [5]:
import openvino_genai as ov_genai
pipe = ov_genai.LLMPipeline(model_dir, "CPU")
print(pipe.generate(user_input, max_new_tokens=1000))

"Imagine a world where humanity has finally reached the stars, and we're not just talking about any old spacecraft, but the most advanced, cutting-edge, and sustainable vessels that are changing the game. Spaceships are not just a luxury, they're a necessity. They're the key to unlocking new frontiers, new discoveries, and new possibilities for humanity. But did you know that the first spaceship was actually a hot air balloon? Yes, you heard that right! In 1783, French inventor Montgolfier created the first successful hot air balloon, which carried a group of 20 people to the skies. It was a groundbreaking achievement that paved the way for the development of modern space travel. Fast forward to today, and we have reusable rockets, advanced propulsion systems, and even private space companies like SpaceX and Blue Origin pushing the boundaries of what's possible. But what's even more exciting is that we're not just talking about the technology, we're talking about the people, the commun